## Imports

In [2]:
# %load_ext autoreload
%reload_ext autoreload
%autoreload 2

from bs4 import BeautifulSoup as BS
import cloudscraper
import os.path
import json
import csv

USE_SUBSET = True

## Get airports and check which ones need different names

In [3]:
with open('utils/airportSubset.csv','r',encoding='utf-8-sig') as csvfile: 
    reader = csv.reader(csvfile, delimiter='\n')
    airports = [x[0] for x in reader]

## Get connecting airports

In [4]:
# We want to webscrape urls that look like this:
# https://www.flightsfrom.com/ATL/destinations?countries=US&durationFrom=44&durationTo=999
scraper = cloudscraper.create_scraper()
connections = {}
if not os.path.isfile("utils/outgoing.json"):
    for airport in airports:
        url = "https://www.flightsfrom.com/" + airport + "/destinations?countries=US&durationFrom=1&durationTo=999"
        r = scraper.get(url)
        soup = BS(r.text, 'html.parser')
        outgoingAirports = soup.find_all(class_ = "airport-font-midheader destination-search-item")
        connections[airport] = []
        for outgoing in outgoingAirports:
            connections[airport].append(str(outgoing).split(" ")[-1][:3])
        if len(connections[airport]) == 0:
            print(airport)
            del connections[airport]
    json.dump(connections,open("utils/outgoing.json","x"))
else:
    connections = json.load(open("utils/outgoing.json"))

## Create Airport Integer Encodings

In [5]:
if not os.path.isfile("utils/airportEncodings.json"):
    airport_to_int = {}
    for i,airport in enumerate(airports):
        airport_to_int[airport] = i
    json.dump(airport_to_int,open("utils/airportEncodings.json","x"))

## Create links to all flight searches

In [13]:
#We want to create urls that look like this:
# "https://www.google.com/travel/flights?q=one+way+flights+from+BLANK+to+BLANK+on+2023-06-09"

if USE_SUBSET:
    with open('utils/airportSubset.csv','r',encoding='utf-8-sig') as csvfile: 
        reader = csv.reader(csvfile, delimiter='\n')
        startAirports = [x[0] for x in reader]
else:
    startAirports = connections.keys()

bad = json.load(open('utils/nameSubstitution.json'))
if not os.path.isfile("utils/flightLinks.csv"):
    f = open("utils/flightLinks.csv","x")
    for start in startAirports:
        # for endAirport in connections[startAirport]:
        for end in startAirports:
            if start == end: continue
            if start in bad: start = bad[start]
            if end in bad: end = bad[end]
            searchURL = "https://www.google.com/travel/flights?q=one+way+flight+from+"+start+"+to+"+end+"+on+"
            f.write(searchURL+"\n")
    f.close()

['LAX', 'SNA', 'ONT', 'ATL', 'TTN', 'PHL', 'EWR', 'JFK', 'SFO', 'OAK', 'RDU']
here
here
LAX
here
LAX
here
LAX
here
LAX
here
LAX
here
LAX
here
LAX
here
LAX
here
LAX
here
LAX
here
SNA
here
here
SNA
here
SNA
here
SNA
here
SNA
here
SNA
here
SNA
here
SNA
here
SNA
here
SNA
here
ONT
here
ONT
here
here
ONT
here
ONT
here
ONT
here
ONT
here
ONT
here
ONT
here
ONT
here
ONT
here
ATL
here
ATL
here
ATL
here
here
ATL
here
ATL
here
ATL
here
ATL
here
ATL
here
ATL
here
ATL
here
Tre
here
Tre
here
Tre
here
Tre
here
Tre
here
Tre
here
Tre
here
Tre
here
Tre
here
Tre
here
Tre
here
PHL
here
PHL
here
PHL
here
PHL
here
PHL
here
here
PHL
here
PHL
here
PHL
here
PHL
here
PHL
here
New
here
New
here
New
here
New
here
New
here
New
here
New
here
New
here
New
here
New
here
New
here
JFK
here
JFK
here
JFK
here
JFK
here
JFK
here
JFK
here
JFK
here
here
JFK
here
JFK
here
JFK
here
SFO
here
SFO
here
SFO
here
SFO
here
SFO
here
SFO
here
SFO
here
SFO
here
here
SFO
here
SFO
here
OAK
here
OAK
here
OAK
here
OAK
here
OAK
here
OAK
here
